In [20]:
import pandas as pd 
import numpy as np

In [21]:
# Charger les fichiers CSV
prod = pd.read_csv('../csv_cleaned/prod_cleaned2.csv')
commandes = pd.read_csv('../csv_cleaned/commandes_cleaned_final.csv')
expeditions = pd.read_csv('../csv_cleaned/expeditions_clean.csv') 
stock = pd.read_csv('../csv_cleaned/stock_nettoye.csv')
retours = pd.read_csv('../csv_cleaned/retours_cleaned_final.csv')

In [22]:
# Convertir la colonne date du stock
stock['date'] = pd.to_datetime(stock['date'])

In [23]:
# Stock le plus récent par produit
latest_stock = stock.sort_values('date').drop_duplicates('id_produit', keep='last')

# Fusion commandes + expéditions
cmd_exped_df = pd.merge(commandes, expeditions, on=["id_commande", "id_produit"], how="left")

# Fusion avec retours
cmd_exped_retours_df = pd.merge(cmd_exped_df, retours, on=["id_commande", "id_produit"], how="left")

# Fusion avec produits
cmd_full_df = pd.merge(cmd_exped_retours_df, prod, on="id_produit", how="left")

# Fusion avec stock
final_df = pd.merge(cmd_full_df, latest_stock[['id_produit', 'niveau_stock', 'entrepot']], on="id_produit", how="left")

In [24]:
#Affichage ou export
final_df.head()  # Pour voir les premières lignes

,id_commande,date_commande,id_produit,quantité_x,entrepôt_x,statut,id_expedition,quantité_y,entrepôt_y,date_expedition,...,mois_x,date_retour,motif,mois_y,catégorie,marque,prix,stock_minimum,niveau_stock,entrepot
0,O000001,2024-09-03,P00413,2,Marseille,Livré,SHP000001,2.0,Marseille,2024-09-09,...,2024-09,NaN,NaN,NaN,Accessoires,Essentielle Paris,151.34,50,89,Paris
1,O000002,2024-11-09,P00778,15,Lyon,Expédié,SHP000002,15.0,Lyon,2024-11-16,...,2024-11,NaN,NaN,NaN,Accessoires,Silhouette Paris,434.14,91,142,Lyon
2,O000004,2024-11-03,P00535,8,Lyon,Livré,SHP000003,8.0,Lyon,2024-11-06,...,2024-11,NaN,NaN,NaN,Maquillage,Soin Absolu,286.57,66,20,Paris
3,O000006,2024-01-24,P00898,3,Paris,Livré,SHP000004,3.0,Paris,2024-01-26,...,2024-01,NaN,NaN,NaN,Soins de la peau,Velours Naturel,90.26,56,149,Lyon
4,O000007,2024-02-15,P00445,1,Lyon,Livré,SHP000005,1.0,Lyon,2024-02-18,...,2024-02,NaN,NaN,NaN,Accessoires,Elixir Haute Beauté,436.94,21,281,Marseille


In [25]:
# (Optionnel) Exporter le résultat dans un nouveau fichier CSV
final_df.to_csv("prod_commandes_expeditions_retours_stocks.csv", index=False)